|<h2>Course:</h2>|<h1><a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">A deep understanding of AI language model mechanisms</a></h1>|
|-|:-:|
|<h2>Part 3:</h2>|<h1>Evaluating LLMs<h1>|
|<h2>Section:</h2>|<h1>Quantitative evaluations<h1>|
|<h2>Lecture:</h2>|<h1><b>CodeChallenge HELPER: Perplexing perplexities<b></h1>|

<br>

<h5><b>Teacher:</b> Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h5>
<h5><b>Course URL:</b> <a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">udemy.com/course/dullms_x/?couponCode=202508</a></h5>
<i>Using the code without the course may lead to confusion or errors.</i>

In [ ]:
# run this code, then restart the python session (and then comment it out)
# !pip install -U datasets huggingface_hub fsspec

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import requests

import torch
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, GPT2Tokenizer
from datasets import load_dataset

# vector plots
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
# load SMALL pretrained GPT-2 model and tokenizer
gpt2 = AutoModelForCausalLM.from_pretrained('gpt2').to(device)
gpt2.eval()

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Exercise 1: A function for perplexity

In [ ]:
def calc_perplex(tokens,model=gpt2,seq_len=1024):

  # number of segments in the total token sequence
  nSegments = torch.numel(tokens) //

  # initialize losses
  sum_losses = 0.

  for i in range(nSegments):

    # find start and end indices
    start =
    end =

    # get the token sequence (with batch dimension)
    X = tokens

    # forward pass
    with torch.no_grad():
      outputs = model(X

    # calculate and store this batch's loss
    sum_losses +=

  # after samples loop, perplexity = exp(average per-token loss)
  perplexity =

  return perplexity

In [ ]:
# generate 50k random tokens (batch=1)
randtokens = torch.randint(tokenizer

# test the perplexity
calc_perplex(randtokens.to(device))

In [ ]:
# the expected perplexity of random tokens equals the vocabulary size (or does it??)
torch.exp(-

# Exercise 2: Perplexities of different texts

In [ ]:
# all books have the same url format; they are unique by numerical code
baseurl='https://www.gutenberg.org/cache/epub/'

bookurls = [
    # code       title
    ['84',    'Frankenstein'    ],
    ['64317', 'GreatGatsby'     ],
    ['11',    'AliceWonderland' ],
    ['1513',  'RomeoJuliet'     ],
    ['76',    'HuckFinn'        ],
    ['219',   'HeartDarkness'   ],
    ['2591',  'GrimmsTales'     ],
    ['2148',  'EdgarAllenPoe'   ],
    ['36',    'WarOfTheWorlds'  ],
    ['829',   'GulliversTravels']
]

In [ ]:
ppls = np.zeros(len(bookurls))

for i,(code,title) in enumerate(bookurls):

  # get the text tokens
  fullurl = baseurl + code + '/pg' + code + '.txt'
  text = requests.get(fullurl).text
  tokens = tokenizer.encode(text,return_tensors='pt')

  # just the first 50k tokens for speed and direct comparison
  tokens = tokens[0][:50000].unsqueeze(0).to(device)

  # calculate perplexity
  ppls[i] =

  print(f'{} has perplexity {}')

In [ ]:
plt.figure(figsize=(10,4))

plt.bar(

plt.show()

# Exercise 3: Perplexities of different models

In [ ]:
text = load_dataset('wikitext','wikitext-2-raw-v1',split='test')
tokens = tokenizer.encode('\n'.join(text['text'][:2000]),return_tensors='pt').to(device)
torch.numel(tokens)

In [ ]:

print(f'The wikitext dataset has perplexity {wiki_per:.3f}')

In [ ]:
# dictionary of modelname:identifier
model_ids = {
    'small':  'gpt2',        # 124M
    'medium': 'gpt2-medium', # 355M
    'large':  'gpt2-large',  # 774M
    'xl':     'gpt2-xl'      # 1.6B
}

# load all models into a dictionary
models = {}
for name, id in model_ids.items():
  models[name] = AutoModelForCausalLM.from_pretrained(id).to(device)
  # switch to eval mode

In [ ]:
perp_models = np.zeros

for i,(name, model) in enumerate(models.items()):
  perp_models[i] =
  print(f'"{name:>8}" has perplexity {perp_models[i]:.2f}')

In [ ]:
plt.figure(figsize=(10,4))

plt.bar(

# Exercise 4: Perplexities with different sequence lengths

In [ ]:
seq_lengths = 2**np.arange(5,11)

perp_by_len = np.zeros(len(seq_lengths))

for i in range(len(seq_lengths)):
  perp_by_len[i] = calc_perplex(tokens
  print(f'Sequence length {seq_lengths[i]:4} has perplexity {perp_by_len[i]:6.2f}')


In [ ]:
# bar plot here :)